<a href="https://colab.research.google.com/github/yunju-1118/ESAA/blob/OB/ESAA_OB_week02_1_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **앙상블 학습과 랜덤 포레스트**
일련의 예측기로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있따. 이때 일련의 예측기를 **앙상블**이라 하고, 이를 **앙상블 학습**이라고 한다.

앙상블은 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 트리 분류기를 훈련시킨다. 예측을 하려면 모든 개별 트리의 예측을 구하면 된다. 그 후, 가장 많은 선택을 받은 클래스를 예측으로 삼는다. 결정 트리의 앙상블을 **랜덤 포레스트**라고 한다.

## **1) 투표 기반 분류기**

더 좋은 분류기를 만드는 간단한 방법은 각 분류기의 예측을 모아 가장 많이 선택된 클래스를 예측하는 것이다. 이렇게 다수결 투표로 정해지는 분류기를 **hard voting** 분류기라고 한다.

이 다수결 투표 분류기가 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다 정확도가 높은 경우가 많다. 이는 **큰 수의 법칙** 때문이다. 약 51%의 정확도를 가진 1,000개의 분류기로 앙상블 모델을 구축한다고 가정했을 때, 가장 많은 클래스를 예측으로 삼는다면 75%의 정확도를 기대할 수 있다. 하지만, 이는 모든 분류기가 완벽히 독립적이고 오차에 상관관계가 없어야 가능하다. 여기서는 같은 데이터로 훈련시키기 때문에 이런 가정이 맞지 않으며, 분류기들이 같은 종류의 오차를 만들기 쉽기 때문에 잘못된 클래스가 다수인 경우가 많고, 앙상블의 정확도가 낮아진다.



In [1]:
import warnings
warnings.filterwarnings('ignore')

# import package
import numpy as np
import os

# 5장에서의 moons dataset 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [4]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.95
RandomForestClassifier 0.95
SVC 1.0
VotingClassifier 0.95


모든 분류기가 클래스의 확률을 예측할 수 있으면, 개별 분류기의 예측을 평균 내어 확률이 ㄱ자ㅏㅇ 높은 클래스를 예측할 수 있따. 이를 **soft voting**이라고 한다. 이 방식은 확률이 높은 투표에 비중을 더 두기 때문에 **hard voting**보다 성능이 높다. SVC는 기본값에서는 클래스 확률을 제공하지 않으므로 probability 매개변수를 True로 지정해야 한다.

## **2)배깅과 페이스팅**
다양한 분류기를 만드는 다른 방법은 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 것이다. 훈련 세트에서 중복을 허용하여 샘플링하는 방식을 **배깅**이라 하며, 중복을 허용하지 않고 샘플링하는 방식을 **페이스팅**이라 한다.

배깅과 페이스팅은 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있으며, 배깅만 한 예측기에서 같은 훈련 샘플을 여러 번 샘플링 할 수 있다.

모든 예측기가 훈련을 마치면 앙상블은 몯느 예측기의 예측을 모아 새로운 샘플에 대한 예측을 만든다. 수집 함수는 전형적으로 분류일 때는 **통계적 최빈값**이고 회귀에 대해서는 **평균**을 계산한다. 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하지만 분산은 줄어든다.

### **2.1) 사이킷런의 배깅과 페이스팅**
사이킷런은 배깅과 페이스팅을 위해 **BaggingClassfier(BaggingRegressor)** 을 제공한다.

다음은 결정 트리 분류기 500개의 앙상블을 훈련시키는 코드이다. 각 분류기는 훈련 세트에서 중복을 허용하여 무작위로 선택된 100개의 샘플로 훈련된다.

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1
)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로 배깅이 페이스팅보다 편향이 조금 더 높다. 하지만, 다양성을 추가한다는 것ㅇ느 예측기들의 상관관계를 줄이므로 앙상블의 분산을 감소시킨다. 전반적으로 배깅이 더 나은 모델을 만들기 때문에 일반적으로 더 선호한다.

### **2.2) oob 평가**
배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링 되고, 어떤 것은 전혀 선택되지 않을 수 있다. BaggingClassifier는 기본값으로 중복을 허용하여(bootstrap=True) 훈련 세트의 크기만큼인 m개 샘플을 선택한다. 이때 선택되지 않은 훈련 샘플의 나머지를 **oob(out-of-bag)** 샘플이라 부른다.

예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않아 별도의 검증 세트를 사용하지 않고 oob 샘플을 이용해 평가할 수 있다.

사이킷런에서 BaggingClassifier를 만들 때, **oob_score=True**로 지정하면 훈련이 끝난 후 자동으로 **oob 평가**를 수행한다.

In [7]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True
)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.95

In [8]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9

oob 샘플에 대한 결정 함수의 값도 oob_decision_function_ 변수에서 확인할 수 있다. 이 경우 결정 함수는 각 훈련 샘플의 클래스 확률을 반환한다.

In [9]:
bag_clf.oob_decision_function_

array([[0.        , 1.        ],
       [0.00531915, 0.99468085],
       [0.6626506 , 0.3373494 ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.82843137, 0.17156863],
       [0.01136364, 0.98863636],
       [0.29444444, 0.70555556],
       [0.65384615, 0.34615385],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.97340426, 0.02659574],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.84090909, 0.15909091],
       [0.96703297, 0.03296703],
       [0.        , 1.        ],
       [0.99465241, 0.00534759],
       [0.        , 1.        ],
       [0.85863874, 0.14136126],
       [0.22651934, 0.77348066],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.08522727, 0.91477273],
       [0.99473684, 0.00526316],
       [1.        , 0.        ],
       [0.

## **3) 랜덤 패치와 랜덤 서브스페이스**
BaggingClassifier는 특성 샘플링도 지원한다. 샘플링은 **max_features**, **bootstrap_features** 두 매개변수로 조절된다. 작동 방식은 max_samples, bootstrap과 동일하지만 특성에 대한 샘플링이다.

이 기법은 특히 매우 고차원의 데이터셋을 다룰 때 유용하다. 훈련 특성과 샘플을 모두 샘플링 하는 것은 **랜덤 패치 방식**이라 한다. 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것을 **서브스페이스 방식**이라 한다.

특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춘다.

## **4)랜덤 포레스트**
랜덤 포레스트는 일반적으로 배깅 방법을 적용한 결정 트리의 앙상블이다. 전형적으로는 max_samples를 훈련 세트의 크기로 지정한다. 결정 트리에 최적화되어 사용하기 편리한 **RandomForestClassifier**를 사용할 수 있다.

In [12]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 후보 중에서 최적의 특성을 찾는 무작위성을 더 주입한다. 이는 결국 트리를 더욱 다양하게 만들고 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만들어낸다.

다음은 BaggingClassifier를 사용해 앞의 RandomForestClassifier와 거의 유사하게 만든 것이다.

In [14]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

### **4.1) 엑스트라 트리**
랜덤포레스트에서 트리를 만들 때 각 노드는 무작위로 서브셋을 만들어 분할에 사용한다. 트리를 더욱 무작위하게 만들기 위해 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그 중에서 최상의 분할을 선택한다.

이와 같이 극단적으로 무작위한 트리의 랜덤 포레스트를 **익스트림 랜덤 트리** 앙상블 (**엑스트라 트리**)라고 부른다. 여기서도 편향이 늘어나지만 분산을 낮추게 되며, 일반적인 랜덤 포레스트보다 엑스트라 트리가 훨씬 빠르다.

엑스트라 트리를 만드려면 사이킷런의 **ExtraTreesClassifier**를 사용한다. 사용법은 **RandomForestClassifier**와 같다.

### **4.2) 특성 중요도**
랜덤 포레스트의 또 다른 장점은 특성의 상대적 중요도를 측정하기 쉽다는 것이다. 사이킷런은 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인해 특성의 중요도를 측정한다.

사이킷런은 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결괏값을 정규화 한다. 이 값은 **feature_importances_** 변수에 저장되어 있다.

In [15]:
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.1005106378719257
sepal width (cm) 0.023945225362607715
petal length (cm) 0.4197958577101689
petal width (cm) 0.4557482790552977


랜덤 포레스트는 특히 **특성을 선택**해야 할 때 어떤 특성이 중요한지 빠르게 확인할 수 있어 매우 편리하다.